# 01. Baseline KPI Analysis

This notebook analyzes baseline KPI measurements from different execution paths:
- NPU-only
- NPU + Fallback
- GPU-only

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from parse_logs import load_log, split_events, calculate_metrics, find_steady_state
from plot_kpi import setup_style, plot_kpi_dashboard, plot_latency_histogram

setup_style()
%matplotlib inline

## 1. Load Data

Place your CSV log files in `../data/` directory.

In [ ]:
DATA_DIR = Path('../data')

# List available log files
log_files = list(DATA_DIR.glob('*.csv'))
print(f"Found {len(log_files)} log files:")
for f in log_files:
    print(f"  - {f.name}")

In [ ]:
# Load a single log file for analysis
# Replace with your actual file name
LOG_FILE = DATA_DIR / 'kpi_log_example.csv'

if LOG_FILE.exists():
    df = load_log(str(LOG_FILE))
    print(f"Loaded {len(df)} records")
    print(f"Duration: {df['elapsed_seconds'].max() / 60:.1f} minutes")
    df.head()

## 2. Overview

In [ ]:
# Split into inference and system events
inference_df, system_df = split_events(df)

print(f"Inference events: {len(inference_df)}")
print(f"System events: {len(system_df)}")

In [ ]:
# Calculate metrics
metrics = calculate_metrics(df)

print("=" * 40)
print("KPI SUMMARY")
print("=" * 40)
print(f"\nLatency:")
print(f"  P50: {metrics.latency_p50:.2f} ms")
print(f"  P95: {metrics.latency_p95:.2f} ms")
print(f"  Mean: {metrics.latency_mean:.2f} ms (±{metrics.latency_std:.2f})")
print(f"  Range: [{metrics.latency_min:.2f}, {metrics.latency_max:.2f}] ms")
print(f"  Count: {metrics.inference_count}")

print(f"\nThermal:")
print(f"  Start: {metrics.thermal_start:.1f} °C")
print(f"  End: {metrics.thermal_end:.1f} °C")
print(f"  Max: {metrics.thermal_max:.1f} °C")
print(f"  Slope: {metrics.thermal_slope:.3f} °C/min")

print(f"\nPower:")
print(f"  Mean: {metrics.power_mean:.1f} mW (±{metrics.power_std:.1f})")

print(f"\nMemory:")
print(f"  Peak: {metrics.memory_peak} MB")

## 3. Visualizations

In [ ]:
# Full dashboard
fig, axes = plot_kpi_dashboard(df, title="Baseline KPI Dashboard")
plt.show()

In [ ]:
# Latency distribution detail
fig, ax = plot_latency_histogram(df, title="Latency Distribution")
plt.show()

## 4. Steady-State Analysis

Exclude warm-up period and analyze steady-state behavior.

In [ ]:
# Extract steady-state (after 30 seconds)
steady_df = find_steady_state(df, warm_up_seconds=30.0)
steady_metrics = calculate_metrics(steady_df)

print("Steady-State Metrics (after 30s warm-up):")
print(f"  Latency P50: {steady_metrics.latency_p50:.2f} ms")
print(f"  Latency P95: {steady_metrics.latency_p95:.2f} ms")
print(f"  Power Mean: {steady_metrics.power_mean:.1f} mW")

## 5. Foreground vs Background (if applicable)

In [ ]:
# Check if we have background data
fg_df = df[df['is_foreground'] == True]
bg_df = df[df['is_foreground'] == False]

print(f"Foreground records: {len(fg_df)}")
print(f"Background records: {len(bg_df)}")

if len(bg_df) > 0:
    fg_metrics = calculate_metrics(fg_df)
    bg_metrics = calculate_metrics(bg_df)
    
    print(f"\nForeground Latency P50: {fg_metrics.latency_p50:.2f} ms")
    print(f"Background Latency P50: {bg_metrics.latency_p50:.2f} ms")

## 6. Export Results

In [ ]:
# Create summary DataFrame
summary = pd.DataFrame([{
    'latency_p50': metrics.latency_p50,
    'latency_p95': metrics.latency_p95,
    'latency_mean': metrics.latency_mean,
    'latency_std': metrics.latency_std,
    'thermal_slope': metrics.thermal_slope,
    'thermal_max': metrics.thermal_max,
    'power_mean': metrics.power_mean,
    'memory_peak': metrics.memory_peak,
    'inference_count': metrics.inference_count,
    'duration_seconds': metrics.duration_seconds
}])

summary